# Osmanlıca Tesseract OCR - Jupyter Notebook Örneği

Bu notebook, Osmanlıca OCR sisteminin temel kullanımını gösterir.

## Kurulum

Önce gerekli paketlerin kurulu olduğundan emin olun:

```bash
pip install -r requirements.txt
```

In [ ]:
# Gerekli kütüphaneleri import et
import sys
import os
sys.path.insert(0, '..')

from scripts.osmanlica_ocr import OsmanlicaOCR
from scripts.preprocess import preprocess_image
import matplotlib.pyplot as plt
from PIL import Image
import cv2

print("✅ Kütüphaneler yüklendi")

## 1. Temel OCR Kullanımı

Basit bir Osmanlıca görüntüden metin çıkarma:

In [ ]:
# OCR nesnesi oluştur
ocr = OsmanlicaOCR()

# Görüntüden metin çıkar
image_path = '../sample-data/images/sample001_besmele.png'
text = ocr.extract_text(image_path)

print(f"Tanınan Metin: {text}")

## 2. Güven Skoru ile OCR

In [ ]:
# Güven skoru ile
text, confidence = ocr.extract_text(image_path, return_confidence=True)

print(f"Tanınan Metin: {text}")
print(f"Güven Skoru: {confidence:.2f}%")

## 3. Görüntü Ön İşleme

In [ ]:
# Orijinal görüntüyü göster
img = Image.open(image_path)
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.imshow(img, cmap='gray')
plt.title('Orijinal Görüntü')
plt.axis('off')

# Ön işleme uygula
processed = preprocess_image(
    image_path,
    '/tmp/processed.png',
    denoise=True,
    binarize=True,
    enhance_contrast=True
)

# İşlenmiş görüntüyü göster
plt.subplot(1, 2, 2)
plt.imshow(processed, cmap='gray')
plt.title('İşlenmiş Görüntü')
plt.axis('off')
plt.tight_layout()
plt.show()

print("✅ Ön işleme tamamlandı")

## 4. Toplu Görüntü İşleme

In [ ]:
# Tüm örnek görüntüleri işle
results = ocr.batch_process(
    '../sample-data/images/',
    '../sample-data/output/'
)

# Sonuçları göster
print("\nİşlenen Görüntüler:")
print("=" * 60)
for filename, text in results.items():
    if text:
        print(f"\n📄 {filename}")
        print(f"   {text[:100]}..." if len(text) > 100 else f"   {text}")
    else:
        print(f"\n❌ {filename}: Hata")
print("\n" + "=" * 60)

## 5. Kelime Konumları

In [ ]:
# Kelime konumlarını al
boxes = ocr.extract_text_with_boxes(image_path)

# Görüntüyü yükle
img = cv2.imread(image_path)
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Her kelime için dikdörtgen çiz
for box in boxes:
    x, y, w, h = box['x'], box['y'], box['width'], box['height']
    cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
# Göster
plt.figure(figsize=(12, 6))
plt.imshow(img_rgb)
plt.title('Kelime Konumları (Yeşil Dikdörtgenler)')
plt.axis('off')
plt.show()

print(f"\n✅ {len(boxes)} kelime tespit edildi")
for i, box in enumerate(boxes[:5], 1):
    print(f"{i}. '{box['text']}' - Güven: {box['confidence']:.1f}%")

## 6. Performans Karşılaştırması

In [ ]:
import time

# Ön işleme olmadan
ocr_no_preprocess = OsmanlicaOCR(preprocess=False)
start = time.time()
text1 = ocr_no_preprocess.extract_text(image_path)
time1 = time.time() - start

# Ön işleme ile
ocr_with_preprocess = OsmanlicaOCR(preprocess=True)
start = time.time()
text2 = ocr_with_preprocess.extract_text(image_path)
time2 = time.time() - start

print("Performans Karşılaştırması:")
print("=" * 60)
print(f"Ön işleme olmadan: {time1:.3f} saniye")
print(f"Ön işleme ile:     {time2:.3f} saniye")
print(f"Fark:              {abs(time2-time1):.3f} saniye")

## 7. Özel Ayarlarla OCR

In [ ]:
# Özel yapılandırma ile OCR
ocr_custom = OsmanlicaOCR(
    language='ara',  # Sadece Arapça
    preprocess=True
)

# Özel config ayarla
ocr_custom.config = '--oem 3 --psm 6'

text = ocr_custom.extract_text(image_path)
print(f"Özel ayarlarla tanınan metin: {text}")

## Sonuç

Bu notebook'ta Osmanlıca Tesseract OCR sisteminin:

1. ✅ Temel kullanımını
2. ✅ Güven skoru hesaplamayı
3. ✅ Görüntü ön işlemeyi
4. ✅ Toplu işlemeyi
5. ✅ Kelime konum tespitini
6. ✅ Performans karşılaştırmayı
7. ✅ Özel yapılandırmayı

gördük.

### Daha Fazla Bilgi

- [README.md](../README.md)
- [API Dokümantasyonu](../docs/API.md)
- [Eğitim Rehberi](../docs/EGITIM.md)
- [Optimizasyon İpuçları](../docs/OPTIMIZASYON.md)